<a href="https://colab.research.google.com/github/rahiakela/small-language-models-fine-tuning/blob/main/hands-on-llm-fine-tuning-with-pytorch-and-huggingface/02_low_rank_adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chapter 3: Low-Rank Adaptation (LoRA)

## Spoilers

In this chapter, we will:

- Understand what a low-rank adapter is and why it’s useful
- Prepare the quantized model for training
- Use `peft` to create and attach adapters to a base model
- Discuss configuration options for targeting layers for training

## Setup

In [1]:
%%capture

# If you're running on Colab
!pip install datasets bitsandbytes trl

In [2]:
# If you're running on runpod.io's Jupyter Template
#!pip install datasets bitsandbytes trl transformers peft huggingface-hub accelerate safetensors pandas matplotlib

In [3]:
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy
from numpy.linalg import matrix_rank
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

## The Goal

We attach adapters to the huge linear layers in an LLM to drastically reduce the number of trainable parameters. We can easily shrink the number of trainable parameters down to less than 1% of their original number. By reducing both computation (fewer gradients to compute) and memory footprint (fewer parameters tracked by the optimizer), we achieve significant efficiency gains. Keep in mind, however, that low-rank adapters are unlikely to match the performance of full-model tuning, and their effectiveness may vary depending on the base model and the task.

The idea behind low-rank adaptation (LoRA) is to train smaller matrices to approximate updates to large
matrices instead of updating them directly. This approach is especially useful for fine-tuning large models, as it minimizes computational and memory overhead.

To better understand its power, you need to be familiar with the basics of matrix multiplication and,
preferably, the idea of decomposing a large matrix into two smaller ones (e.g. singular value decomposition,
SVD).

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/matmul.png?raw=True)
<center>Figure 3.1 - Matrix multiplication</center>


Notice that the two small matrices have nine elements altogether, while the resulting matrix has twenty. Even
in a tiny example such as this, the total number of elements was reduced in 55% (from twenty to nine). The
greater the ratio between the "remaining" and "facing" dimensions, the larger the reduction.

The resulting matrix, although 4 by 5, has redundancy that allows it to be represented by smaller matrices. The
redundancy is measured by the matrix’s rank, which counts the number of independent rows or columns. In
our example, the rank is determined by the "facing" dimensions of the small matrices, which is one.

## Low-Rank Adaptation in a Nutshell

The rank shows you the real dimensions of the matrix—or basically, how much redundancy is in
there:

- A high-ranking value (that is, one close to the actual number of rows or columns) means there’s little to no
redundancy.
- A low-rank matrix implies there’s a lot of redundancy, and the large matrix can be easily represented by
two smaller ones.

Now, you know why it’s called a low-rank adapter. We’re using two small matrices to produce a low-rank big
matrix (representing the updates) that matches the size of the original layer.

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/two_matrices.png?raw=True)
<center>Figure 3.2 - Multiplying two low-rank matrices</center>

Let’s start with a base layer as if it were one of the linear layers in the attention module. We’re making it a big,
square, one million-weight matrix:

In [4]:
base_layer = nn.Linear(1024, 1024, bias=False)
base_layer.weight.shape, base_layer.weight.numel()

(torch.Size([1024, 1024]), 1048576)

We’d need one million updates to apply to these one million weights.

So, let’s create two small matrices:

- The first one, let’s call it layer A, has to match the input features of the base layer.
- The second one, let’s call it layer B, has to match the output features of the base layer.
- Since they need to be multiplied, their inner ("facing") dimensions must match:
  - This is the rank of the matrix that will result from their multiplication, and you get to choose this value.



![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/lowrank_matrices.png?raw=True)
<center>Figure 3.3 - Frozen weights and low-rank matrices</center>

If our rank of choice is eight, our layers look like this:

In [5]:
torch.manual_seed(11)

r = 8
layer_A = nn.Linear(base_layer.in_features, r, bias=False)
layer_B = nn.Linear(r, base_layer.out_features, bias=False)

In [6]:
layer_A.weight.shape, layer_B.weight.shape

(torch.Size([8, 1024]), torch.Size([1024, 8]))

In [7]:
layer_A, layer_B

(Linear(in_features=1024, out_features=8, bias=False),
 Linear(in_features=8, out_features=1024, bias=False))

In [8]:
layer_A.weight.numel(), layer_B.weight.numel()

(8192, 8192)

Nonetheless, if we multiply them, we get a full-sized 1M-parameter matrix as a result:

In [9]:
composite = layer_B.weight @ layer_A.weight
composite.shape, composite.numel()

(torch.Size([1024, 1024]), 1048576)

If we call Numpy’s matrix_rank() function, however, it will expose its true dimensions (its rank):

In [10]:
matrix_rank(composite.detach().numpy())

np.int64(8)

This is what you described, as it would happen inside a layer’s
forward() method.

$$
\Large
\text{output} = X @ (W + B @ A)^T
$$
<center>Equation 3.1 - Adding the resulting product to the weights</center>

In [11]:
torch.manual_seed(19)

batch = torch.randn(1, 1024)

batch @ (base_layer.weight.data + layer_B.weight @ layer_A.weight).T

tensor([[-0.4918, -0.1138, -0.8948,  ..., -0.8239,  0.1980,  0.7781]],
       grad_fn=<MmBackward0>)

But, thanks to the distributive property of matrix multiplication, we can split this into two passes: one forward
pass through the base layer, and another through the resulting low-rank matrix.

$$
\Large
\text{output} = \underbrace{X @ W^T}_{O_W} + \underbrace{X @ (B @ A)^T}_{O_{AB}}
$$
<center>Equation 3.2 - Using two forward passes</center>

This is very convenient and easy to implement since we can keep the original flow (on the left of the following
figure) and compute an additional output using our two small matrices. In the end, we simply add them
together:

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch3/forward.png?raw=True)
<center>Figure 3.4 - Using two forward passes</center>

In [12]:
regular_output = batch @ base_layer.weight.data.T
additional_output = batch @ (layer_B.weight @ layer_A.weight).T
regular_output, additional_output

(tensor([[-0.4561, -0.1368, -0.4342,  ..., -0.5319,  0.0037,  0.2741]]),
 tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
        grad_fn=<MmBackward0>))

In [13]:
regular_output.shape, additional_output.shape

(torch.Size([1, 1024]), torch.Size([1, 1024]))

As it turns out, the additional output can be split into two chained operations—that is, passing the input batch
through layer A, and then its output through layer B.

$$
\Large
\text{additional} = X @ (B @ A)^T = \underbrace{\underbrace{(X @ A^T)}_{O_A} @ B^T}_{O_{AB}}
$$
<center>Equation 3.3 - Chaining the adapter’s forward passes</center>

This equivalence is easy to verify in code.

In [14]:
out_A = (batch @ layer_A.weight.T)
additional_output = out_A @ layer_B.weight.T
additional_output

tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
       grad_fn=<MmBackward0>)

So, if we stick with the layers themselves, it’s as straightforward as this:

In [15]:
regular_output = base_layer(batch)
out_A = layer_A(batch)
additional_output = layer_B(out_A)
output = regular_output + additional_output
regular_output, additional_output, output

(tensor([[-0.4561, -0.1368, -0.4342,  ..., -0.5319,  0.0037,  0.2741]],
        grad_fn=<MmBackward0>),
 tensor([[-0.0357,  0.0230, -0.4607,  ..., -0.2920,  0.1944,  0.5041]],
        grad_fn=<MmBackward0>),
 tensor([[-0.4918, -0.1138, -0.8948,  ..., -0.8239,  0.1980,  0.7781]],
        grad_fn=<AddBackward0>))

In [16]:
regular_output.shape, additional_output.shape, output.shape

(torch.Size([1, 1024]), torch.Size([1, 1024]), torch.Size([1, 1024]))

The final output matches the first output we had obtained!

There’s just one final, small detail: the additional output can be adjusted using a multiplier.

The higher the
multiplier, the greater the impact of the adapters. By convention, the multiplier is not a single number but a
fraction, as shown in the equation below.

$$
\Large
\text{output} = X @ W^T + \frac{\alpha}{r}\left[X @ (B @ A)^T\right]
$$
<center>Equation 3.4 - LoRA’s alpha</center>

The denominator is fixed and it’s the rank we chose, but we can tweak the numerator, which is called LoRA’s
alpha.

In practice, alpha’s often set to twice the rank, effectively doubling the additional output.

In [17]:
alpha = 2*r
output = regular_output + (alpha / r) * additional_output
output

tensor([[-0.5275, -0.0908, -1.3555,  ..., -1.1160,  0.3924,  1.2822]],
       grad_fn=<AddBackward0>)

In [18]:
output.shape

torch.Size([1, 1024])

## The Road So Far

In [19]:
supported = torch.cuda.is_bf16_supported(including_emulation=False)
compute_dtype = (torch.bfloat16 if supported else torch.float32)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=compute_dtype
)

# Here is our 4-bit quantized model
model_q4 = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-350m",
    device_map='cuda:0',
    torch_dtype=compute_dtype,
    quantization_config=nf4_config
)

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## Parameter Types and Gradients

****
**Summary of "Parameter Types and Gradients"**
- quantization only freezes the linear layers that have been quantized
- after quantization, a model can be prepared using the `prepare_model_for_kbit_training()` function
  - it freezes **all** layers
  - it casts every non-quantized 16-bit layer to FP32 to improve training
  - it enables gradient checkpointing
- you'll be able to unfreeze layers of your choice later on using the LoRA configuration
****

In [20]:
def trainable_parms(model):
    parms = [(name, param.dtype) for name, param in model.named_parameters() if param.requires_grad]
    return parms

trainable_parms(model_q4.model)

[('decoder.embed_tokens.weight', torch.float32),
 ('decoder.embed_positions.weight', torch.float32),
 ('decoder.layers.0.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.0.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.0.final_layer_norm.weight', torch.float32),
 ('decoder.layers.0.final_layer_norm.bias', torch.float32),
 ('decoder.layers.1.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.1.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.1.final_layer_norm.weight', torch.float32),
 ('decoder.layers.1.final_layer_norm.bias', torch.float32),
 ('decoder.layers.2.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.2.self_attn_layer_norm.bias', torch.float32),
 ('decoder.layers.2.final_layer_norm.weight', torch.float32),
 ('decoder.layers.2.final_layer_norm.bias', torch.float32),
 ('decoder.layers.3.self_attn_layer_norm.weight', torch.float32),
 ('decoder.layers.3.self_attn_layer_norm.bias', torch.float32),
 ('decoder.la

Actually, there’s one more thing we need to do, and that’s preparing the model or, more precisely, calling the
`prepare_model_for_kbit_training()` method.

In [21]:
prepared_model = prepare_model_for_kbit_training(
    model_q4,
    use_gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False}
)
prepared_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
      (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (

This method wraps the entire protocol for preparing a model before running a training. This includes:

- Casting the layer norms to FP32.
- Making the output of the embedding layer require grads.
- Add the upcasting of the lm_head to FP32.
- Freezes the whole model, meaning that none of its trainable layers can be trained anymore.


How many trainable parameters does it currently have?

In [22]:
trainable_parms(prepared_model)

[]

Next, let’s use a new helper function, parms_of_dtype(), to get the names of all the layers whose parameters
are of a certain type.

In [23]:
def parms_of_dtype(model, dtype=torch.float32):
    parms = [name for name, param in model.named_parameters() if param.dtype == dtype]
    return parms

In [24]:
parms_of_dtype(prepared_model)

['model.decoder.embed_tokens.weight',
 'model.decoder.embed_positions.weight',
 'model.decoder.layers.0.self_attn.k_proj.bias',
 'model.decoder.layers.0.self_attn.v_proj.bias',
 'model.decoder.layers.0.self_attn.q_proj.bias',
 'model.decoder.layers.0.self_attn.out_proj.bias',
 'model.decoder.layers.0.self_attn_layer_norm.weight',
 'model.decoder.layers.0.self_attn_layer_norm.bias',
 'model.decoder.layers.0.fc1.bias',
 'model.decoder.layers.0.fc2.bias',
 'model.decoder.layers.0.final_layer_norm.weight',
 'model.decoder.layers.0.final_layer_norm.bias',
 'model.decoder.layers.1.self_attn.k_proj.bias',
 'model.decoder.layers.1.self_attn.v_proj.bias',
 'model.decoder.layers.1.self_attn.q_proj.bias',
 'model.decoder.layers.1.self_attn.out_proj.bias',
 'model.decoder.layers.1.self_attn_layer_norm.weight',
 'model.decoder.layers.1.self_attn_layer_norm.bias',
 'model.decoder.layers.1.fc1.bias',
 'model.decoder.layers.1.fc2.bias',
 'model.decoder.layers.1.final_layer_norm.weight',
 'model.decode

How much memory footprint?

In [25]:
prepared_model.get_memory_footprint()/1e6

264.15104

## *Parameter-Efficient Fine-Tuning(PEFT)*


"_🤗 PEFT (Parameter-Efficient Fine-Tuning) is a library for efficiently adapting large pretrained models to various downstream applications without fine-tuning all of a model’s parameters because it is prohibitively costly. PEFT methods only fine-tune a small number of (extra) model parameters - significantly decreasing computational and storage costs - while yielding performance comparable to a fully fine-tuned model. This makes it more accessible to train and store large language models (LLMs) on consumer hardware._

_PEFT is integrated with the Transformers, Diffusers, and Accelerate libraries to provide a faster and easier way to load, train, and use large models for inference._"

****
**Summary of "PEFT"**
- the basic configuration below should work well in many cases
```python
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model, config)
```
- ranks of 8, 16, or 32 are typical, but using higher values shouldn’t significantly impact the model’s memory footprint.
- the scaling factor, `lora_alpha` is typically twice the rank.
- if your model has `Conv1D` layers, add `fan_in_fan_out=True` to your configuration
- if your model was recently released, you may need to specify the `target_modules` manually
  - typically, use the names of the massive linear layers in the attention module.
- by default, only the adapters are trainable
  - if you'd like to train other layers, such as layer norms, add them to the `modules_to_save` argument
  - if you're adding your own tokens to the tokenizer, you'll need to also train vocabulary-related layers such as embeddings and the model's head
****

In [26]:
lora_config = LoraConfig()
lora_config

LoraConfig(task_type=None, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, peft_version='0.18.0', base_model_name_or_path=None, revision=None, inference_mode=False, r=8, target_modules=None, exclude_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None, arrow_config=None, ensure_weight_tying=False)

So, the typical—and
minimal—configuration looks like this:

In [27]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

**`target_modules`**

Since there are new models and architectures being released on a weekly basis, chances are that there is no preconfigured list of target layers in your currently installed version of the PEFT library. In this case, you’ll be greeted with the following error:

***
`ValueError: Please specify `target_modules` in `peft_config``
***

Once you have the names, you can use yet another configuration argument: target_modules, which is either
the name or a list of the names of the modules to which you want to apply the adapters.

**Supported Models**

If you'd like to check if a given model's architecture is already supported by the installed version of the `peft` package, you can do the following:

In [28]:
from peft.utils.constants import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING.keys()

dict_keys(['t5', 'mt5', 'bart', 'gpt2', 'bloom', 'blip-2', 'opt', 'gptj', 'gpt_neox', 'gpt_neo', 'bert', 'roberta', 'xlm-roberta', 'electra', 'deberta-v2', 'deberta', 'layoutlm', 'llama', 'llama4', 'chatglm', 'gpt_bigcode', 'mpt', 'RefinedWebModel', 'RefinedWeb', 'falcon', 'btlm', 'codegen', 'mistral', 'mixtral', 'stablelm', 'phi', 'gemma', 'gemma2', 'gemma3_text', 'qwen2', 'qwen3', 'rwkv', 'rwkv7'])

Let's see the supported models for Phi.

In [29]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['phi']

['q_proj', 'v_proj', 'fc1', 'fc2']

Let's see the supported models for llama4.

In [30]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['llama4']

['q_proj', 'v_proj']

Let's see the supported models for qwen3.

In [31]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['qwen3']

['q_proj', 'v_proj']

## The PEFT Model

At this point, though, we’re doing it ourselves to better understand how the model is effectively modified. It’s
pretty easy:

In [32]:
peft_model = get_peft_model(prepared_model, config, adapter_name='default')
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 512, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
          (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
          (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_fea

In [33]:
TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['opt']

['q_proj', 'v_proj']

Let’s take a closer look at the modified layer:

In [34]:
lin = peft_model.base_model.model.model.decoder.layers[0].self_attn.k_proj
lin

Linear4bit(in_features=1024, out_features=1024, bias=True)

In [35]:
lin = peft_model.base_model.model.model.decoder.layers[0].self_attn.q_proj
lin

lora.Linear4bit(
  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=1024, out_features=8, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=8, out_features=1024, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
  (lora_magnitude_vector): ModuleDict()
)

In [36]:
lin = peft_model.base_model.model.model.decoder.layers[0].self_attn.v_proj
lin

lora.Linear4bit(
  (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=1024, out_features=8, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=8, out_features=1024, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
  (lora_magnitude_vector): ModuleDict()
)

In [37]:
fc = peft_model.base_model.model.model.decoder.layers[0].fc1
fc

Linear4bit(in_features=1024, out_features=4096, bias=True)

In [38]:
fc = peft_model.base_model.model.model.decoder.layers[0].fc2
fc

Linear4bit(in_features=4096, out_features=1024, bias=True)

Do you wonder how many trainable parameters our model has now?

In [39]:
peft_model.print_trainable_parameters()

trainable params: 786,432 || all params: 331,982,848 || trainable%: 0.2369


The trainable parameters are—all of them—from the attached adapters: they are layers A (named lora_A) and
B (named lora_B).

In [40]:
trainable_parms(peft_model.base_model.model)

[('model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.0.self_attn.q_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.1.self_attn.q_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.v_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.v_proj.lora_B.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.q_proj.lora_A.default.weight',
  torch.float32),
 ('model.decoder.layers.2.self_attn.q_proj.lora_B.default.weight'

**`modules_to_save`**

The good news is that you can pick and
choose which layers to unfreeze (e.g., layer norms, embeddings, the head).

In [41]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['layer_norm'] # unfreezing layer norms
)

In [42]:
# Since the model is modified in-place, we need to unload adapters
# from previous configuration to avoid mixing them.
# In a regular workflow, you'd load configuration only once and
# this wouldn't be needed.
_ = peft_model.unload()

In [44]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(


trainable params: 884,736 || all params: 332,081,152 || trainable%: 0.2664


Keeping the layer norms trainable added roughly 100,000 parameters that need to be trained.

## Embeddings

If you need to add new tokens to the tokenizer’s vocabulary, you may need to resize your embedding layers and model’s head.

In [49]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    modules_to_save=['layer_norm', 'embed_tokens'] # unfreezing embedding layers as well
)

In [53]:
# Since the model is modified in-place, we need to unload adapters
# from previous configuration to avoid mixing them.
# In a regular workflow, you'd load configuration only once and
# this wouldn't be needed.
_ = peft_model.unload()

In [54]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

trainable params: 26,624,000 || all params: 383,559,680 || trainable%: 6.9413


The embedding layer alone has approximately 26 million parameters, roughly 7% of the base model’s total
size.

Let’s add it to the list of target
modules instead:

In [55]:
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['embed_tokens', 'q_proj', 'v_proj']
)

In [56]:
# Since the model is modified in-place, we need to unload adapters
# from previous configuration to avoid mixing them.
# In a regular workflow, you'd load configuration only once and
# this wouldn't be needed.
_ = peft_model.unload()

In [58]:
peft_model = get_peft_model(prepared_model, config)
peft_model.print_trainable_parameters()

trainable params: 1,192,704 || all params: 358,128,384 || trainable%: 0.3330


Check it out: the embedding layer uses the lora_embedding_A and lora_embedding_B attributes.

In [59]:
lin = peft_model.base_model.model.model.decoder.embed_tokens
lin

lora.Embedding(
  (base_layer): Embedding(50272, 512, padding_idx=1)
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.05, inplace=False)
  )
  (lora_A): ModuleDict()
  (lora_B): ModuleDict()
  (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 8x50272 (cuda:0)])
  (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 512x8 (cuda:0)])
  (lora_magnitude_vector): ModuleDict()
)

## Managing Adapters

In [ ]:
peft_model.load_adapter('dvgodoy/opt-350m-lora-yoda', adapter_name='yoda')
lora_A = peft_model.base_model.model.model.decoder.layers[0].self_attn.q_proj.lora_A
lora_A

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [ ]:
peft_model.add_adapter(adapter_name='third', peft_config=config)
lora_A

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
  (third): Linear(in_features=1024, out_features=8, bias=False)
)

In [ ]:
peft_model.delete_adapter(adapter_name='third')
lora_A

ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [ ]:
peft_model.peft_config.keys()

dict_keys(['default', 'yoda'])

In [ ]:
peft_model.active_adapter

'default'

In [ ]:
peft_model.set_adapter('yoda')
peft_model.active_adapter

'yoda'

```python
with peft_model.disable_adapter():
    original_outputs = peft_model(inputs)

original_outputs = peft_model.base_model(inputs)
```

In [ ]:
peft_model.merge_adapter(adapter_names=['yoda'])
lora_A

  warnings.warn(


ModuleDict(
  (default): Linear(in_features=1024, out_features=8, bias=False)
  (yoda): Linear(in_features=1024, out_features=8, bias=False)
)

In [ ]:
peft_model.unload()
peft_model.base_model.model.model.decoder.layers[0].self_attn

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 512, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
          (project_out): Linear4bit(in_features=1024, out_features=512, bias=False)
          (project_in): Linear4bit(in_features=512, out_features=1024, bias=False)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
              )
              (activation_fn): ReLU()
              (self_attn_layer_norm): LayerNorm((1024,

### Coming Up in "Fine-Tuning LLMs"

Low-rank adapters saved the day by swooping in and enabling fast and cheap fine-tuning for LLMs. These humongous models, although powerful, are masters of a single trade—predicting the next token—thus remaining limited by the structure of their inputs. A new kind of input must be developed to enable these creatures to chat. Learn more about the incredible tale of chat templates in the next chapter of "Fine-Tuning LLMs."